In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import LinearSVC,SVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
import pickle

In [2]:
train_data=pd.read_csv("products_sentiment_train.tsv",header=None,sep="\t")

In [3]:
train_data.head()

,0,1
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [56]:
X_train=train_data[0]
Y_train=train_data[1]

In [5]:
test_data=pd.read_csv("products_sentiment_test.tsv",sep="\t")

In [6]:
test_data.head()

,Id,text
0,0,"so , why the small digital elph , rather than ..."
1,1,3/4 way through the first disk we played on it...
2,2,better for the zen micro is outlook compatibil...
3,3,6 . play gameboy color games on it with goboy .
4,4,"likewise , i 've heard norton 2004 professiona..."


In [7]:
X_test=test_data["text"]

In [8]:
test_id=test_data["Id"]

Проверка accuracy для различных комбинаций векторайзеров и классификаторов на кросс-валидации

In [9]:
CV=5
lr_pipeline=Pipeline([("vectorizer",CountVectorizer(ngram_range=(1,2))),
                      ("classifier",LogisticRegression())])
scores=cross_val_score(lr_pipeline,X_train,Y_train,scoring="accuracy",cv=CV)
print scores.mean()

0.770504443778


In [10]:
lr_pipeline=Pipeline([("vectorizer",CountVectorizer()),("classifier",SGDClassifier())])
scores=cross_val_score(lr_pipeline,X_train,Y_train,scoring="accuracy",cv=CV)
print scores.mean()

C:\Users\dim\Anaconda2\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.744996881231


In [11]:
lr_pipeline=Pipeline([("vectorizer",TfidfVectorizer()),("classifier",LogisticRegression())])
scores=cross_val_score(lr_pipeline,X_train,Y_train,scoring="accuracy",cv=CV)
print scores.mean()

0.766503184395


In [12]:
lr_pipeline=Pipeline([("vectorizer",CountVectorizer()),("classifier",LinearSVC())])
scores=cross_val_score(lr_pipeline,X_train,Y_train,scoring="accuracy",cv=CV)
print scores.mean()

0.754000653129


In [13]:
et_pipeline=Pipeline([("vectorizer",CountVectorizer(ngram_range=(1,2))),
                      ("classifier",ExtraTreesClassifier(n_estimators=300))])
scores=cross_val_score(lr_pipeline,X_train,Y_train,scoring="accuracy",cv=CV)
print scores.mean()

0.754000653129


In [14]:
lr_pipeline=Pipeline([("vectorizer",CountVectorizer(ngram_range=(3,5),
                                                    analyzer="char_wb")),
                      ("classifier",LogisticRegression())])
scores=cross_val_score(lr_pipeline,X_train,Y_train,scoring="accuracy",cv=CV)
print scores.mean()

0.759496887481


In [15]:
vectorizer=CountVectorizer(ngram_range=(3,5),analyzer="char_wb")
clf=MultinomialNB(alpha=2.0)
X=vectorizer.fit_transform(X_train)
scores=cross_val_score(clf,X,Y_train,scoring="accuracy",cv=CV)
print scores.mean()

0.785009468809


Применение выбранного классификатора

In [16]:
lr_pipeline=Pipeline([("vectorizer",CountVectorizer(ngram_range=(1,2))),
                      ("classifier",LogisticRegression())])

In [17]:
lr_pipeline.fit(X_train,Y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
    ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [18]:
with open("pipeline.pkl","wb") as pipeline_pickle:
     pickle.dump(lr_pipeline,pipeline_pickle)

In [19]:
predict=lr_pipeline.predict(X_test)

In [20]:
pr_proba=lr_pipeline.predict_proba(X_test)

In [21]:
pr_proba

array([[  2.06940704e-02,   9.79305930e-01],
       [  8.21359357e-01,   1.78640643e-01],
       [  1.29192485e-01,   8.70807515e-01],
       [  3.42637846e-01,   6.57362154e-01],
       [  6.22297365e-01,   3.77702635e-01],
       [  6.40802026e-01,   3.59197974e-01],
       [  2.60652597e-03,   9.97393474e-01],
       [  9.00148949e-01,   9.98510510e-02],
       [  9.74723201e-01,   2.52767990e-02],
       [  1.31415754e-01,   8.68584246e-01],
       [  6.00306642e-02,   9.39969336e-01],
       [  8.40405285e-01,   1.59594715e-01],
       [  7.53717839e-02,   9.24628216e-01],
       [  6.02436096e-01,   3.97563904e-01],
       [  9.58159920e-02,   9.04184008e-01],
       [  2.62248906e-01,   7.37751094e-01],
       [  3.55353248e-01,   6.44646752e-01],
       [  7.23429194e-01,   2.76570806e-01],
       [  3.30693274e-02,   9.66930673e-01],
       [  4.43642719e-01,   5.56357281e-01],
       [  2.15018238e-02,   9.78498176e-01],
       [  5.51879439e-02,   9.44812056e-01],
       [  

In [22]:
sub=pd.DataFrame()
sub["Id"]=test_id
sub["y"]=predict

In [23]:
sub.to_csv("sub.csv",index=False)

In [24]:
from sklearn.externals import joblib

In [25]:
model=joblib.load("pipeline.pkl")

In [26]:
prediction=model.predict(X_test)

In [27]:
prediction

array([1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 0,

In [29]:
X_test

0      so , why the small digital elph , rather than ...
1      3/4 way through the first disk we played on it...
2      better for the zen micro is outlook compatibil...
3        6 . play gameboy color games on it with goboy .
4      likewise , i 've heard norton 2004 professiona...
5      mine was 2 weeks old and i chucked it in the t...
6      i find it very stable and comfortable to use ,...
7      styling / ergonomics : the keys are small , wh...
8      at first i thought it is only a isolated incid...
9                              - light , compact design 
10     although the sd500 takes great quality photos ...
11     after years with that carrier 's expensive pla...
12     i 've only had this camera for a few days , bu...
13     the user support service isn 't too great eith...
14     the headphones have great sound and durability...
15     file transfers are fast , nearly a song per se...
16     creative are * the * sound people for computer...
17     unfortunately , i sold i

In [30]:
print(model)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
    ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])


In [32]:
model.predict(X_test)

array([1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 0,

In [35]:
vectorizer=CountVectorizer(ngram_range=(1,2))
vectorizer.fit(X_train)

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [36]:
with open("vectorizer.pkl","wb") as vectorizer_pickle:
     pickle.dump(vectorizer,vectorizer_pickle)

In [39]:
model=LogisticRegression()
X_train=vectorizer.transform(X_train)
model.fit(X_train,Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [40]:
with open("model.pkl","wb") as model_pickle:
     pickle.dump(model,model_pickle)

In [41]:
vectorizer=joblib.load("vectorizer.pkl")
model=joblib.load("model.pkl")

In [52]:
prediction

array([1], dtype=int64)

In [53]:
X_test[0]

"so , why the small digital elph , rather than one of the other cameras with better resolution or picture quality ? size [ + 2 ] # # because , unless it 's small , i won 't cary it around ."

In [61]:
Y_train[Y_train==0]

3       0
12      0
14      0
15      0
20      0
21      0
22      0
23      0
24      0
26      0
27      0
30      0
34      0
39      0
41      0
44      0
46      0
50      0
57      0
58      0
66      0
67      0
71      0
77      0
78      0
80      0
81      0
83      0
84      0
85      0
       ..
1917    0
1919    0
1925    0
1926    0
1929    0
1930    0
1933    0
1935    0
1938    0
1939    0
1941    0
1942    0
1944    0
1956    0
1957    0
1958    0
1961    0
1968    0
1974    0
1975    0
1978    0
1980    0
1981    0
1982    0
1983    0
1986    0
1993    0
1994    0
1996    0
1999    0
Name: 1, Length: 726, dtype: int64

In [62]:
X_train[1957]

'after scouring the various boards i find this is an arbitrary and pervasive problem .'